In [33]:
import pandas as pd 
import numpy as np 
import tensorflow as tf


In [34]:
df=pd.read_csv("C:\\Users\\saipr\\amazon_review\\data\\amazon.csv")

In [35]:
df.head()

,Text,label
0,This is the best apps acording to a bunch of ...,1
1,This is a pretty good version of the game for ...,1
2,this is a really . there are a bunch of levels...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1


In [36]:
df.shape

(19996, 2)

In [37]:
df['label'].value_counts()

label
1    15230
0     4766
Name: count, dtype: int64

In [38]:
X=df['Text']
y=df['label']

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.layers import LSTM,GRU,Bidirectional,Dense,Dropout
from tensorflow.keras.regularizers import L1,L2 
from tensorflow.keras.callbacks import EarlyStopping 

In [40]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X)

In [41]:
vocab=len(tokenizer.word_index)+1

In [42]:
vocab

19402

In [43]:
sequences = tokenizer.texts_to_sequences(X)


In [44]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(sequences, y, test_size=0.25, random_state=42)


In [48]:
X_train[2]

[6,
 8,
 100,
 90,
 92,
 1001,
 662,
 368,
 2,
 2719,
 33,
 86,
 4,
 261,
 7,
 501,
 212,
 10,
 26,
 335,
 2,
 778,
 10,
 85,
 12,
 4077,
 1,
 610,
 6,
 263,
 968,
 12,
 88,
 328,
 13,
 11683,
 18,
 3,
 258,
 176,
 11684,
 5,
 1,
 53,
 126,
 1245,
 17,
 72,
 1,
 4460]

In [49]:
max_length = max(len(seq) for seq in sequences)
print("Maximum sequence length:", max_length)


Maximum sequence length: 62


In [50]:
sequence_lengths = [len(seq) for seq in sequences]

print("Max length:", max(sequence_lengths))
print("Min length:", min(sequence_lengths))
print("Average length:", sum(sequence_lengths) / len(sequence_lengths))


Max length: 62
Min length: 1
Average length: 34.29885977195439


In [51]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Padding training and testing sequences
X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post')

print("Shape of X_train_padded:", X_train_padded.shape)
print("Shape of X_test_padded:", X_test_padded.shape)


Shape of X_train_padded: (14997, 62)
Shape of X_test_padded: (4999, 62)


In [53]:
print(vocab)
print(max_length)

19402
62


In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define parameters
vocab_size = 19402  # Total vocabulary size
max_length = 62      # Maximum sequence length
embedding_dim = 100  # Embedding vector size (can be tuned)

# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))  # Embedding Layer
model.add(GRU(32))  # LSTM Layer with 100 units
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()


c:\Users\saipr\amazon_review\venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [81]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (can also use 'val_accuracy')
    patience=3,          # Number of epochs with no improvement before stopping
    restore_best_weights=True,  # Restore model weights from the best epoch
    verbose=1
)



In [82]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define your class labels
class_labels = np.unique(y_train)  # Get unique class labels (e.g., [0, 1])

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=y_train)

# Convert to dictionary format for Keras
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

print("Class Weights:", class_weight_dict)


Class Weights: {0: np.float64(2.1146362098138747), 1: np.float64(0.6548336389834949)}


In [83]:
model.fit(X_train_padded, y_train, 
          epochs=10, 
          batch_size=32, 
          validation_data=(X_test_padded, y_test), 
          class_weight=class_weight_dict,callbacks=[early_stopping])  # <-- Add class weights here


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - accuracy: 0.5373 - loss: 0.6970 - val_accuracy: 0.7560 - val_loss: 0.6825
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.7478 - loss: 0.5931 - val_accuracy: 0.8848 - val_loss: 0.2783
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.9138 - loss: 0.2343 - val_accuracy: 0.8834 - val_loss: 0.2844
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9573 - loss: 0.1165 - val_accuracy: 0.9012 - val_loss: 0.2560
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 40ms/step - accuracy: 0.9802 - loss: 0.0681 - val_accuracy: 0.8980 - val_loss: 0.3210
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9909 - loss: 0.0354 - val_accuracy: 0.8900 - val_loss: 0.3854
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 16s 35ms/step - accuracy: 0.9918 - loss: 0.0264 - val_accuracy: 0.8898 - val_loss: 0.4298
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 4.


In [84]:
model.evaluate(X_test_padded,y_test)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9016 - loss: 0.2556


[0.2559770941734314, 0.9011802077293396]

In [85]:
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test_padded) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
              precision    recall  f1-score   support

           0       0.79      0.82      0.80      1220
           1       0.94      0.93      0.93      3779

    accuracy                           0.90      4999
   macro avg       0.86      0.87      0.87      4999
weighted avg       0.90      0.90      0.90      4999



In [87]:
new_review = ["This product is Bad!"]
new_seq = tokenizer.texts_to_sequences(new_review)
new_padded = pad_sequences(new_seq, maxlen=62, padding='post')
prediction = model.predict(new_padded)
print("Positive" if prediction > 0.5 else "Negative")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Negative


In [89]:
# Save the model in HDF5 format
model.save("sentiment_model.h5")


In [90]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model("sentiment_model.h5")


In [92]:
new_review = ["This product is Good!"]
new_seq = tokenizer.texts_to_sequences(new_review)
new_padded = pad_sequences(new_seq, maxlen=62, padding='post')
prediction = loaded_model.predict(new_padded)
print("Positive" if prediction > 0.5 else "Negative")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Positive


In [94]:
import joblib

# Save the tokenizer
joblib.dump(tokenizer, "tokenizer.pkl")

# Save class weights if needed
joblib.dump(class_weight_dict, "class_weights.pkl")


['class_weights.pkl']